.

Data_Engineering_TIL_(20191104)

### 1. 실습개요

이 실습에서는 Amazon EMR 을 사용하여 Google 도서의 Ngram 을 SQL을 이용하여 분석한다. 

"The sun rises in the East and sets in the West." 문장을 예로들면 이 문장의 일부 2 그램은 "the sun", "in the" 및 "sets in "이다. 3그램은 "sets in the "이고 샘플 4 그램 은 "rises in the east" 이다. N-그램은 특정 단어가 순서대로 나타날 확률을 예측하는 데 사용된다. N-그램은 웹 페이지 및 휴대폰에서 입력 제안을 제공하는 데 유용 할 수 있습니다.


### 2. 실습목표

- Hive 및 Ganglia 를 실행하는 Amazon EMR 클러스터 생성


- Hive 문을 사용하여 Amazon S3 에 저장된 Google Ngram 입력 데이터에서 테이블 생성


- Hive 쿼리를 실행하여 데이터 드릴 다운 및 분석


- Ganglia 를 사용하여 AMR 클러스터 모니터링

### 3. 실습 상세내용

먼저 HIVE를 포함하는 EMR 클러스터 생성한다.

그 다음에 Hive 접속 및 로우데이터 접근한다.

아래 그림과 같이 hive에 접속하고, 명령어들을 실행해준다.

이 섹션에서는 로우 Ngrams 데이터에 액세스한다. 1 그램 에 대한 데이터에 액세스하게 되는데, 이는 모든 책에서 발견 된 단일 단어의 수를 제공합니다. 이 데이터에는 2 억 6,400 만 개의 항목이 있으며 2.6GB 의 디스크 저장소를 차지한다. 

Ngram 데이터는 Amazon S3 에서 액세스 할 수 있으며 hive에서 외부 테이블을 생성하여 Amazon EMR 에서 직접 액세스 할 수 있습니다 . 이 정의는 Amazon EMR 에 데이터의 형식 및 위치를 알려준다.

중간에 DESCRIBE ngrams; 명령어를 날리게 되면 아래 그림과 같은 결과를 볼 수 있는데 이는 생성한 테이블의 구조에 대한 설명을 불러오는 명령어이다.

- gram 은 n-gram 의 단어 (또는 단어 집합)


- year 는 n-gram 이 출판물에 나타난 해


- occurrences 은 주어진 연도에 n- 그램이 나타난 횟수의 개수


- pages 는 주어진 연도에 n-gram 이 나타난 페이지 수


- book 은 주어진 해에 n- 그램이 나타난 책의 수

중간에 SELECT * FROM ngrams LIMIT 10; 명령어와 같이 SQL SELECT 문을 사용하여 데이터의 샘플을 볼 수 있는 명령어를 실행할 수 있다.

<img src="3.png" width="1000" />

불행히도 현재 형식의 데이터는 매우 원시적 인 것으로 보입니다. 첫 번째 열(gram)에는 문장 부호와 기타 특수 문자 (예 : #)가 있으며 대문자와 소문자가 모두 존재한다. 데이터를 분석하기에는 지저분하기 때문에 분석하기 전에 데이터를 깨끗한 형식으로 정규화 해야한다.

그래서 위에 그림과 같이 CREATE TABLE normalized (gram string, year int, occurrences bigint); 명령어를 실행하여 분석하는 사람이 관심있는 열만으로 정규화 결과를 저장할 테이블을 생성하는 명령어를 실행한다.

그래서 다음과 같이 정규화 된 데이터의 결과를 저장할 새 테이블을 만들어지게 된다.

- 3 자 미만의 단어 제외


- 알파벳이 아닌 문자가 포함 된 단어 제외 (하이픈 및 아포스트로피는 허용 가능)


- 모든 문자를 소문자로 변환

그런 다음으로 ngrams 테이블 에서 데이터를 선택한 다음 새 정규화 된 테이블에 삽입한다. 

쿼리는

- 1990 년에서 2005 년 사이의 데이터 만 사용


- n- 그램을 소문자로 변환


- 3 자 이상의 단어 만 포함


-  알파벳 문자, 아포스트로피 및 하이픈이 포함 된 단어 만 포함

따라서 아래와 같이 명령어를 실행하여 ngrams 테이블에서 데이터를 선택하고, 정규화 된 테이블에 삽입한다.

In [ ]:
INSERT OVERWRITE TABLE normalized
SELECT lower(gram), year, occurrences
FROM ngrams
WHERE year BETWEEN 1990 AND 2005
AND gram REGEXP "^[A-Za-z+\'-]{3,}$";

<img src="4.png" width="1000" />

위에 그림에서 SELECT * FROM normalized LIMIT 20; 명령어 실행결과를 보면 

책에 표시된 연도 및 해당 연도의 단어 발생 횟수와 함께 단어가 표시되는 것을 볼 수 있습니다.

분석가가 모든 책에서 가장 인기있는 단어를 보는 데 관심이 있다면 아래 그림과 같은 명령어를 실행하여 몇 년 동안 모든 책에서 가장 많이 사용 된 50개의 단어를 추출할 수 있다.

<img src="5.png" width="1000" />

아래 그림과 같은 명령어를 실행하여 가장 많이 사용되는 50 개의 단어를 10 자 이상으로 추출해본다.

<img src="6.png" width="1000" />

분석가는 이제 단어사용이 년마다 어떻게 다른지에 관심이 있을 수 있습니다. 이를 검사하는 가장 쉬운 방법 은 해당 연도에 저장된 총 단어 수와 비교하여 각 단어의 상대적 발생 비율 을 포함하는 새 테이블을 만드는 것이다.

예를 들어, 워드클라우드가 1년에 10,000 번, 내년에 13,000 번 나타나는 경우, 단어가 인기를 얻었는지 또는 더 많은 책이 데이터베이스에 포함되어 있는지 확실하지 않다. 비율을 계산하면 주어진 연도의 다른 모든 단어와 비교하여 단어 의 상대적 빈도를 비교할 수 있다.

아래 그림과 같이 CREATE TABLE ratios (gram string, year int, occurrences bigint, ratio double); 명령어를 실행하여 비율 테이블을 생성한 후에 아래와 같이 명령어를 실행하여 비율 테이블을 정규화 된 테이블 에서 계산 된 데이터로 채운다.

In [ ]:
NSERT OVERWRITE TABLE ratios
SELECT
 a.gram,
 a.year,
sum(a.occurrences) AS occurrences,
 sum(a.occurrences) / b.total AS ratio
FROM normalized a
JOIN (SELECT year, sum(occurrences) AS total
 FROM normalized
 GROUP BY year) b ON (a.year = b.year)
GROUP BY a.gram, a.year, b.total;

<img src="7.png" width="1000" />

이 명령문은 내부조인 을 사용하여 같은 해에 사용 된 모든 단어에 대한 단어 발생 비율을 계산한다.
이 문장은 실행하는 데 약 3 분이 걸린다. 추가적으로 Ganglia 를 사용하여 클러스터를 모니터링 할 수 있습니다. 
이제 매년 및 단어에 대해 전년도의 비율 변화를 쿼리 할 수 있다. 이것은 연도 간의 값을 비교하기 위해 비율 테이블을 자체에 결합하여 수행된다.

아래 그림과 같은 명령어를 실행하여 매년 비율의 차이를 계산해본다. 이 문장은 매우 드문 단어에 대해 급격한 증가를 피하기 위해 1000 개 미만의 단어로 모든 단어를 필터링한다.

<img src="8.png" width="1000" />

그러면 다음 질문사항에 대한 쿼리를 날려보자.

- 1999 년 가장 인기있는 단어는 무엇입니까?


- 인터넷 이라는 단어의 인기 증가는 어떻습니까?

아래 그림과 같이 쿼리를 날려 결과를 확인할 수 있다.

<img src="9.png" width="1000" />

결과는 월드 와이드 웹이 발명 된 1994/1995 년 경에 인터넷 사용이 폭발적으로 증가했음을 보여준다. 

마지막으로, 약간의 재미를 위해 아래 그림과 같이 쿼리를 실행하여 2000 년 동안 각 길이의 가장 인기있는 단어를 찾아본다.

<img src="10.png" width="1000" />

** 추가적으로 EMR 클러스터 띄울때 강글리아를 선택해주면 콘솔에서 접속하여 아래 그림과 같이 클러스터의 리소스 모니터링을 할 수 있다.

<img src="11.png" width="1000" />